# Finite Difference Stencils

We want to represent $f'(0)$ in the following form (say up to $O(h^5)$):
$$
f'(0) = \frac{c_1 f(-2h) + c_2 f(-h) + c_3 f(+h) + c_4 f(+2h)}{h}
$$
which should be ture for any analytic function, for the first few bases, we can choose:
$$
\begin{align}
f(x) = 1, f(x) = x, f(x) = x^2, f(x) = x^3.
\end{align}
$$
Take derivatives
$$
\begin{align}
\left.
\begin{array}{c}
\frac{c_1 + c_2 + c_3 + c_4}{h} &= 0,\\
\frac{c_1 (-2h) + c_2 (-h) + c_3 (h) + c_4 (h)}{h} &= 1,\\
\frac{c_1 (-2h)^2 + c_2 (-h)^2 + c_3 (h)^2 + c_4 (2h)^2}{h} &= 0,\\
\frac{c_1 (-2h)^3 + c_2 (-h)^3 + c_3 (h)^3 + c_4 (2h)^3}{h} &= 0,\\
\end{array}
\right\}
%\begin{array}{c}
%
%\end{array}
\end{align}
$$

In [ ]:
include("stencils.jl")
include("../writeio.jl")

In [ ]:
using .FDStencils
using .WriteIO

In [ ]:
simple = [-3//2, -1//2, 1//2, 3//2];
values = [0, 1, 0, 0];

In [ ]:
cs = GetCoefficient(simple, values)

#### Write to Files

In [ ]:
filename = "test.hxx"
thornname = "WaveToyHigherOrderX"

In [ ]:
pr()

In [ ]:
function headpart(pr::Function)
    pr("#ifndef " * replace(uppercase(filename), "."=>"_"))
    pr("#define " * replace(uppercase(filename), "."=>"_"))
    pr()
end

In [ ]:
function bodypart(pr::Function)
    pr("#include <cctk.h>")
    pr("namespace " * thornname * " {")
    pr()
    pr(cs[1])
    pr(cs[2])
end

In [ ]:
function tailpart(pr::Function)
    pr("} // namespace " * thornname)
    pr("#endif // #ifndef " * replace(uppercase(filename), "."=>"_"))
end

## Write to Files

In [ ]:
# filename = "test.hxx"
# thornname = "WaveToyHigherOrderX"
# if isfile(filename)
#     println("file ", filename, " already, exist, replacing it ...")
#     rm(filename)
# end
# filepointer = open(filename, "a")
# pr(x="") = println(filepointer, x)

In [ ]:
# headpart();
# bodypart();
# tailpart();
# println("Done generating ", filename)
# close(filepointer)

In [11]:
WriteFile(headpart, bodypart, tailpart, filename)

file test.hxx already, exist, replacing it ...


LoadError: UndefVarError: `pr` not defined